In [1]:
import cv2
import numpy as np
import socket
import pickle
import struct
import mediapipe as mp
import time

# Khởi tạo MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Khởi tạo MediaPipe Drawing Utilities
mp_drawing = mp.solutions.drawing_utils

# Khởi tạo socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 123))
server_socket.listen(10)

conn, addr = server_socket.accept()
print(f"Đã kết nối với client tại địa chỉ: {addr}")

data = b""
payload_size = struct.calcsize("L")
send_count = 0

while True:
    start_time = time.time()

    while len(data) < payload_size:
        try:
            data += conn.recv(65536)
        except Exception as e:
            print(f"Không thể nhận dữ liệu: {e}")
            break
    
    if len(data) < payload_size:
        print("Dữ liệu không đủ")
        break

    packed_msg_size = data[:payload_size]
    data = data[payload_size:]
    msg_size = struct.unpack("L", packed_msg_size)[0]
    
    while len(data) < msg_size:
        data += conn.recv(65536)
    
    frame_data = data[:msg_size]
    data = data[msg_size:]
    
    frame = pickle.loads(frame_data)

    # Xoay frame 90 độ về bên trái
    frame = cv2.transpose(frame)
    frame = cv2.flip(frame, 0)

    # Sử dụng MediaPipe để lấy keypoints
    results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Vẽ keypoints lên hình ảnh
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Hiển thị hình ảnh
    cv2.imshow('Image from Machine 1', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Tính toán FPS
    end_time = time.time()
    fps = 1 / (end_time - start_time)
    result = f"FPS: {fps:.2f}"

    if send_count // 20 == 0:
        try:
            conn.sendall(result.encode())  # Gửi kết quả trở lại máy 1
        except Exception as e:
            print(f"Không thể gửi dữ liệu: {e}")
            break

        send_count = 0

    send_count += 1

    time.sleep(0.1)

cv2.destroyAllWindows()


In [2]:
import cv2
import numpy as np
import socket
import struct
import mediapipe as mp
import threading
import time

# Khởi tạo MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Khởi tạo MediaPipe Drawing Utilities
mp_drawing = mp.solutions.drawing_utils

# Khởi tạo socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 123))
server_socket.listen(10)

conn, addr = server_socket.accept()
print(f"Đã kết nối với client tại địa chỉ: {addr}")

def receive_data():
    data = b""
    payload_size = struct.calcsize("L")
    while True:
        start_time = time.time()
        
        while len(data) < payload_size:
            data += conn.recv(4096)

        packed_msg_size = data[:payload_size]
        data = data[payload_size:]
        msg_size = struct.unpack("L", packed_msg_size)[0]

        while len(data) < msg_size:
            data += conn.recv(4096)

        frame_data = data[:msg_size]
        data = data[msg_size:]
        frame = np.frombuffer(frame_data, dtype=np.uint8)
        frame = cv2.imdecode(frame, cv2.IMREAD_COLOR)
        # Xoay frame 90 độ về bên trái
        frame = cv2.transpose(frame)
        frame = cv2.flip(frame, 0)

        h, w, _ = frame.shape

        results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        frame = cv2.resize(frame, (w // 2, h // 2))
        
        cv2.imshow('Image from Raspberry Pi', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        # Tính toán FPS
        end_time = time.time()
        fps = 1 / (end_time - start_time)
        result = f"FPS: {fps:.2f}".encode()

        # Gửi FPS về Raspberry Pi
        conn.sendall(result)

threading.Thread(target=receive_data).start()


Đã kết nối với client tại địa chỉ: ('192.168.1.11', 45148)


Exception in thread Thread-6 (receive_data):
Traceback (most recent call last):
  File "c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\PC\AppData\Local\Temp\ipykernel_20860\867467544.py", line 31, in receive_data
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
